In [7]:
from dotenv import load_dotenv
load_dotenv()
import os 
import google.generativeai as genai
genai.configure(api_key=os.environ.get('GEMINI_API_KEY'))
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

In [104]:
from pinecone import Pinecone
pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))
pc.create_index(
  name="rag",
  dimension=768,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
)

In [ ]:
import json 
data = json.load(open("reviews.json"))
data['reviews']

In [ ]:
processed_data = []

for review in data['reviews']: 
    result = genai.embed_content(
    model="models/text-embedding-004",
    content=review['review'],
    task_type="retrieval_document",
    title="Embedding of single string")
    embedding = result['embedding']
    processed_data.append({
        "values": embedding, 
        "id": review["professorName"], 
        "metadata": {
            "review": review["review"], 
            "subject": review["subject"], 
            "stars": review["starRating"]
        }
    })


print(processed_data)

In [105]:
from pinecone import Pinecone

pc = Pinecone(api_key=os.environ.get("PINECONE_RAG_KEY"))
index = pc.Index("rag")
index.upsert(
    vectors=processed_data, 
    namespace="ns1"
)

{'upserted_count': 20}

In [107]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}